<a href="https://colab.research.google.com/github/chenweioh/notebooks/blob/main/Biopython_read_a_fasta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FASTA located at https://d396qusza40orc.cloudfront.net/genpython/data_sets/dna2.fasta

In [7]:
# Install Biopython
!pip install biopython

# Import necessary libraries
from google.colab import files
from Bio import SeqIO

# Prompt user to upload a file
uploaded = files.upload()

# Assuming the user uploads a single file, read its contents
# Note: This example assumes the uploaded file is in FASTA format. Adjust accordingly for different file types.
for filename in uploaded.keys():
    print(f"User uploaded file '{filename}' with length {len(uploaded[filename])} bytes")
    # Read the file using Biopython (adjust the format as necessary, e.g., 'fasta', 'genbank')
    for record in SeqIO.parse(filename, "fasta"):
        print(f"Record: {record.id}, Length: {len(record.seq)}")
        print(f"Sequence: {record.seq}\n")

# Note: If there could be multiple files or different file types, you'll need to add logic to handle those scenarios.


Saving dna2.fasta to dna2.fasta
User uploaded file 'dna2.fasta' with length 48593 bytes
Record: gi|142022655|gb|EQ086233.1|91, Length: 4635
Sequence: CTCGCGTTGCAGGCCGGCGTGTCGCGCAACGACGTGTGGGGCCTGACGGGCAGGGAGGATCTCGGCGGCGCCAACTATGCGGTCTTTCGGCTCGAAAGCCAGTTCCAGACCTCCGACGGCGCGCTGACCGTGCCCGGCTCCGCATTCAGTTCGCAAGCCTACGTCGGGCTCGGCGGCGACTGGGGGACCGTGACGCTCGGGCGCCAGTTCGATTTCGTCGGCGATCTGATGCCGGCTTTCGCGATCGGCGCGAACACGCCGGCCGGCCTGCTCGCGTGGGGCTTGCCGGCGAATGCGTCGGCGGGCGGTGCGCTCGACAACCGCGTGTGGGGCGTCCAGGTGAACAATGCGGTGAAGTACGTGAGCCCGACGTTCGGCGGATTGTCGTTCGGCGGCCTGTGGGGCTTCGGCAACGTGCCCGGCACGGTCGCGCGCAGCAGCGTGCAAAGCGCGATGCTGTCCTACACGCAAGGCGCGTTCAGCGCCGCGCTCGCTTATTTCGGCCAGCACGATGTAACTGCCGGTGGCAATCTGCGCAATTTCTCGGGCGGTGCAGGCTACAACGTCGGGCAGTTCCGCGTCTTCGGCATGGTGTCGGACGTGCGGATCAGCGCCGCCGCGCCGCTGCGGGCCACGACCTATGACGGCGGCTTGACCTATGCGGTCACGCCGGCGTTGCAGCTCGGCGGCGGCTTCCAGTACCAGCAGCGCGGCGGCGACATCGGCTCGGCCAACCAGGTCACGTTGAGCGCCGACTATTCGCTGTCGAAGCGTACCGGCCTTTACGTGGTATTCGCACGCGGGCACGACAGTGCGTATGGCGCGCAGGTCGAGGCGGCGCTCGGCGGGG

In [8]:
# Import necessary library from Bio
from Bio import SeqIO

# Initialize a variable to count the records
record_count = 0

# Loop through each file uploaded in the previous step
for filename in uploaded.keys():
    # Use SeqIO to parse the FASTA file and count the records
    record_count += sum(1 for record in SeqIO.parse(filename, "fasta"))

# Display the total number of records found in the file
print(f"Total number of records in the file: {record_count}")


Total number of records in the file: 18


In [9]:
# Initialize a list to store the lengths of all sequences and a dictionary to map lengths to sequence identifiers
sequence_lengths = []
length_to_ids = {}

# Loop through each file uploaded in the previous step
for filename in uploaded.keys():
    # Parse the FASTA file
    for record in SeqIO.parse(filename, "fasta"):
        # Append the sequence length to the list
        seq_length = len(record.seq)
        sequence_lengths.append(seq_length)
        # Update the dictionary to map sequence length to its identifiers
        if seq_length not in length_to_ids:
            length_to_ids[seq_length] = [record.id]
        else:
            length_to_ids[seq_length].append(record.id)

# Calculate the longest and shortest sequence lengths
longest_seq_length = max(sequence_lengths)
shortest_seq_length = min(sequence_lengths)

# Identify the identifiers of the longest and shortest sequences
longest_seq_ids = length_to_ids[longest_seq_length]
shortest_seq_ids = length_to_ids[shortest_seq_length]

# Display the results
print(f"Lengths of sequences: {sequence_lengths}")
print(f"Longest sequence length: {longest_seq_length}, Identifiers: {longest_seq_ids}")
print(f"Shortest sequence length: {shortest_seq_length}, Identifiers: {shortest_seq_ids}")

# Check and print if there's more than one longest or shortest sequence
if len(longest_seq_ids) > 1:
    print(f"There are {len(longest_seq_ids)} sequences sharing the longest length.")
else:
    print("There is only one longest sequence.")

if len(shortest_seq_ids) > 1:
    print(f"There are {len(shortest_seq_ids)} sequences sharing the shortest length.")
else:
    print("There is only one shortest sequence.")


Lengths of sequences: [4635, 1151, 4894, 3511, 4076, 2867, 442, 890, 967, 4338, 1352, 4564, 4804, 964, 2095, 1432, 115, 2646]
Longest sequence length: 4894, Identifiers: ['gi|142022655|gb|EQ086233.1|255']
Shortest sequence length: 115, Identifiers: ['gi|142022655|gb|EQ086233.1|346']
There is only one longest sequence.
There is only one shortest sequence.


To identify open reading frames (ORFs) within sequences from a FASTA file based on a given input reading frame, and to extract information about the longest ORF across the file as well as within specific sequences, the following Python program can be utilized. This program will:

Accept an input reading frame (1, 2, or 3) for the forward strand.
Find all ORFs for each sequence in the FASTA file.
Determine the longest ORF across the entire file and within specific sequences.
Provide the identifier of the sequence containing the longest ORF.
For a given sequence identifier, return the longest ORF and its starting position.

In [10]:
from Bio import SeqIO
from Bio.Seq import Seq
import io

# Define the functions for finding ORFs
def find_orfs(sequence, frame):
    start_codon = 'ATG'
    stop_codons = {'TAA', 'TAG', 'TGA'}
    orfs = []
    seq_len = len(sequence)

    for i in range(frame - 1, seq_len, 3):
        if sequence[i:i+3] == start_codon:
            for j in range(i+3, seq_len, 3):
                if sequence[j:j+3] in stop_codons:
                    orfs.append((i+1, j+3))  # Store the start and end positions
                    break  # Move to the next potential start codon
    return orfs

def analyze_orfs(uploaded_files, frame):
    longest_orf_length = 0
    longest_orf_seq_id = None
    longest_orf_start = 0

    for filename in uploaded_files.keys():
        # Parse the FASTA file from the uploaded content
        for record in SeqIO.parse(io.StringIO(uploaded_files[filename].decode()), 'fasta'):
            sequence = str(record.seq)
            orfs = find_orfs(sequence, frame)

            for start_pos, end_pos in orfs:
                orf_length = end_pos - start_pos + 1
                if orf_length > longest_orf_length:
                    longest_orf_length = orf_length
                    longest_orf_seq_id = record.id
                    longest_orf_start = start_pos

    # Output results
    print(f"Longest ORF length: {longest_orf_length}")
    print(f"Identifier of sequence with longest ORF: {longest_orf_seq_id}")
    print(f"Starting position of the longest ORF: {longest_orf_start}")

# Assume 'uploaded' is the dictionary containing the uploaded file(s) from the previous steps
frame = 1  # Example frame
analyze_orfs(uploaded, frame)


Longest ORF length: 2394
Identifier of sequence with longest ORF: gi|142022655|gb|EQ086233.1|45
Starting position of the longest ORF: 385


In [11]:
from Bio import SeqIO
import io
from collections import defaultdict

def find_repeats(uploaded_files, n):
    # Dictionary to count occurrences of each repeat
    repeat_counts = defaultdict(int)

    # Iterate over each uploaded file
    for filename in uploaded_files.keys():
        # Read sequences from the file
        for record in SeqIO.parse(io.StringIO(uploaded_files[filename].decode()), 'fasta'):
            sequence = str(record.seq)
            # Slide over the sequence to find repeats of length n and count them
            for i in range(len(sequence) - n + 1):
                repeat = sequence[i:i+n]
                repeat_counts[repeat] += 1

    # Determine the most frequent repeat(s) and its frequency
    max_frequency = max(repeat_counts.values())
    most_frequent_repeats = [repeat for repeat, count in repeat_counts.items() if count == max_frequency]

    # Display the results
    print(f"Repeats of length {n} and their counts:")
    for repeat, count in repeat_counts.items():
        print(f"{repeat}: {count}")

    print(f"\nMost frequent repeat(s) of length {n}: {most_frequent_repeats} with a frequency of {max_frequency}")

# Example usage
n = 3  # Length of the repeat to search for
find_repeats(uploaded, n)


Repeats of length 3 and their counts:
CTC: 604
TCG: 1737
CGC: 2538
GCG: 2551
CGT: 1235
GTT: 485
TTG: 392
TGC: 1134
GCA: 1091
CAG: 846
AGG: 459
GGC: 1618
GCC: 1659
CCG: 1778
CGG: 1726
GTG: 808
TGT: 385
GTC: 916
CAA: 374
AAC: 511
ACG: 1224
CGA: 1790
GAC: 944
TGG: 572
GGG: 558
CCT: 485
CTG: 836
TGA: 533
GGA: 519
GAG: 589
GAT: 898
ATC: 897
TCT: 295
CCA: 583
ACT: 242
CTA: 109
TAT: 157
ATG: 586
GGT: 621
CTT: 377
TTT: 219
TTC: 676
GCT: 902
GAA: 708
AAA: 222
AAG: 369
AGC: 911
AGT: 250
TCC: 539
AGA: 299
ACC: 650
CCC: 560
CAT: 611
ATT: 266
TCA: 522
TAC: 296
ACA: 365
CAC: 729
AAT: 258
GTA: 282
TTA: 60
TAA: 57
TAG: 118
ATA: 176

Most frequent repeat(s) of length 3: ['GCG'] with a frequency of 2551


**What is the length of the longest ORF appearing in reading frame 2 of any of the sequences?**


To calculate the length of the longest open reading frame (ORF) appearing in reading frame 2 of any of the sequences in your uploaded FASTA file, you can use a modified version of the previous ORF-finding script. This script specifically targets reading frame 2 and calculates the lengths of all ORFs found, ultimately identifying the longest one.



In [12]:
from Bio import SeqIO
import io

def find_orfs_in_frame(sequence, frame):
    start_codon = 'ATG'
    stop_codons = {'TAA', 'TAG', 'TGA'}
    longest_orf_length = 0  # Initialize the longest ORF length

    # Adjust the sequence for the specified frame
    adjusted_sequence = sequence[frame-1:]

    orf_start = None
    for i in range(0, len(adjusted_sequence), 3):
        codon = adjusted_sequence[i:i+3]
        if orf_start is None and codon == start_codon:
            orf_start = i
        elif orf_start is not None and codon in stop_codons:
            orf_length = i + 3 - orf_start
            if orf_length > longest_orf_length:
                longest_orf_length = orf_length
            orf_start = None  # Reset for the next ORF

    return longest_orf_length

def find_longest_orf(uploaded_files, frame):
    overall_longest_orf = 0

    for filename in uploaded_files.keys():
        # Parse the FASTA file
        for record in SeqIO.parse(io.StringIO(uploaded_files[filename].decode()), 'fasta'):
            longest_orf = find_orfs_in_frame(str(record.seq), frame)
            if longest_orf > overall_longest_orf:
                overall_longest_orf = longest_orf

    print(f"The length of the longest ORF in reading frame {frame} is: {overall_longest_orf}")

# Set the frame to 2 as per the user's request
frame = 2
find_longest_orf(uploaded, frame)


The length of the longest ORF in reading frame 2 is: 1458


Question 5
What is the starting position of the longest ORF in reading frame 3 in any of the sequences? The position should indicate the character number where the ORF begins. For instance, the following ORF:

> sequence1

ATGCCCTAG

starts at position 1.

In [15]:
from Bio import SeqIO
import io

def find_longest_orf_with_start(sequence, frame):
    start_codon = 'ATG'
    stop_codons = {'TAA', 'TAG', 'TGA'}
    longest_orf_length = 0
    longest_orf_start = None

    # Adjust for the specific frame
    adjusted_sequence = sequence[frame-1:]

    current_orf_start = None
    for i in range(0, len(adjusted_sequence), 3):
        codon = adjusted_sequence[i:i+3]
        if current_orf_start is None and codon == start_codon:
            current_orf_start = i
        elif current_orf_start is not None and codon in stop_codons:
            orf_length = i + 3 - current_orf_start
            if orf_length > longest_orf_length:
                longest_orf_length = orf_length
                longest_orf_start = current_orf_start + frame  # Adjust back to original sequence indexing
            current_orf_start = None

    # Return both length and starting position (None if no ORF found)
    return longest_orf_length, longest_orf_start

def analyze_longest_orf_start(uploaded_files, frame):
    overall_longest_orf_length = 0
    overall_longest_orf_start = None

    for filename in uploaded_files.keys():
        # Parse each sequence from the uploaded files
        for record in SeqIO.parse(io.StringIO(uploaded_files[filename].decode()), 'fasta'):
            longest_orf_length, longest_orf_start = find_longest_orf_with_start(str(record.seq), frame)
            if longest_orf_length > overall_longest_orf_length:
                overall_longest_orf_length = longest_orf_length
                overall_longest_orf_start = longest_orf_start

    # Print the starting position of the longest ORF in the specified frame
    if overall_longest_orf_start is not None:
        print(f"The starting position of the longest ORF in reading frame {frame} is: {overall_longest_orf_start}")
    else:
        print(f"No ORF found in reading frame {frame}.")

# Set the frame to 3 as per the user's request
frame = 3
analyze_longest_orf_start(uploaded, frame)


The starting position of the longest ORF in reading frame 3 is: 636


What is the length of the longest forward ORF that appears in the sequence with the identifier  gi|142022655|gb|EQ086233.1|16?



In [16]:
from Bio import SeqIO
import io

def find_longest_orf_in_all_frames(sequence):
    start_codon = 'ATG'
    stop_codons = {'TAA', 'TAG', 'TGA'}
    longest_orf_length = 0

    # Check all three forward reading frames
    for frame in range(3):
        current_orf_start = None
        for i in range(frame, len(sequence), 3):
            codon = sequence[i:i+3]
            if current_orf_start is None and codon == start_codon:
                current_orf_start = i
            elif current_orf_start is not None and codon in stop_codons:
                orf_length = i + 3 - current_orf_start
                if orf_length > longest_orf_length:
                    longest_orf_length = orf_length
                current_orf_start = None  # Reset for the next ORF

    return longest_orf_length

def find_longest_orf_for_identifier(uploaded_files, identifier):
    for filename in uploaded_files.keys():
        # Parse the FASTA file
        for record in SeqIO.parse(io.StringIO(uploaded_files[filename].decode()), 'fasta'):
            if record.id == identifier:
                longest_orf_length = find_longest_orf_in_all_frames(str(record.seq))
                print(f"The length of the longest forward ORF for {identifier} is: {longest_orf_length}")
                return

    print("Identifier not found in the uploaded files.")

# Example usage
identifier = 'gi|142022655|gb|EQ086233.1|16'
find_longest_orf_for_identifier(uploaded, identifier)


The length of the longest forward ORF for gi|142022655|gb|EQ086233.1|16 is: 1644


Find the most frequently occurring repeat of length 6 in all sequences. How many times does it occur in all?



In [17]:
from Bio import SeqIO
import io
from collections import defaultdict

def find_most_frequent_repeat(uploaded_files, repeat_length):
    repeat_counts = defaultdict(int)

    # Iterate through each uploaded file
    for filename in uploaded_files.keys():
        # Parse the sequences from the file
        for record in SeqIO.parse(io.StringIO(uploaded_files[filename].decode()), 'fasta'):
            sequence = str(record.seq)
            # Identify and count all repeats of the specified length
            for i in range(len(sequence) - repeat_length + 1):
                repeat = sequence[i:i+repeat_length]
                repeat_counts[repeat] += 1

    # Determine the most frequent repeat and its count
    most_frequent_repeat, max_count = max(repeat_counts.items(), key=lambda x: x[1])

    # Output the results
    print(f"The most frequently occurring repeat of length {repeat_length} is: '{most_frequent_repeat}'")
    print(f"It occurs {max_count} times in all sequences.")

# Example usage
repeat_length = 6
find_most_frequent_repeat(uploaded, repeat_length)


The most frequently occurring repeat of length 6 is: 'GCGCGC'
It occurs 153 times in all sequences.


Question 9
Find all repeats of length 12 in the input file. Let's use Max to specify the number of copies

of the most frequent repeat of length 12.  How many different 12-base sequences

occur Max times?

In [20]:
from Bio import SeqIO
import io
from collections import defaultdict

def find_repeats_max_occurrences(uploaded_files, repeat_length):
    repeat_counts = defaultdict(int)

    # Iterate through each uploaded file
    for filename in uploaded_files.keys():
        # Parse the sequences from the file
        for record in SeqIO.parse(io.StringIO(uploaded_files[filename].decode()), 'fasta'):
            sequence = str(record.seq)
            # Identify and count all repeats of the specified length
            for i in range(len(sequence) - repeat_length + 1):
                repeat = sequence[i:i+repeat_length]
                repeat_counts[repeat] += 1

    # Find the maximum occurrence frequency
    max_occurrences = max(repeat_counts.values())

    # Count how many different sequences occur this max number of times
    max_repeats_count = sum(1 for count in repeat_counts.values() if count == max_occurrences)

    # Output the results
    print(f"Number of different 12-base sequences that occur {max_occurrences} times (Max): {max_repeats_count}")

# Example usage
repeat_length = 12
find_repeats_max_occurrences(uploaded, repeat_length)


Number of different 12-base sequences that occur 10 times (Max): 4


Question 10
Which one of the following repeats of length 7 has a maximum number of occurrences?

In [19]:
repeat_length = 7
find_most_frequent_repeat(uploaded, repeat_length)

The most frequently occurring repeat of length 7 is: 'CGCGCCG'
It occurs 63 times in all sequences.


# To find out what species an unknown DNA sequence comes from using Biopython
, you can perform a BLAST search against a nucleotide database and analyze the top hits. This process involves using the Bio.Blast.NCBIWWW module to run an online BLAST search and the Bio.Blast.NCBIXML module to parse the results.

1. Performing a BLAST Search:
First, perform a BLAST search with the unknown DNA sequence. This will be done using NCBI's online BLAST service through Biopython's NCBIWWW module.

In [21]:
from Bio.Blast import NCBIWWW
sequence = "TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTACAATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCACCTACGGTAGAG"
result_handle = NCBIWWW.qblast("blastn", "nt", sequence)


**Parsing the BLAST Results:**
After performing the BLAST search, you can parse the results using the NCBIXML module. This will allow you to extract information about the top hits, such as the species names.

This script performs a BLAST search with your sequence against the NCBI nucleotide database and then parses and prints details of the top 5 hits, including the sequence title (which often contains the species name), the alignment length, the e-value (a measure of the significance of the hit), and the number of identities (matching bases).

Please note that running this script requires an internet connection and might take some time to complete the BLAST search, depending on the current load on NCBI's servers.

Since you're working with an unknown sequence, reviewing the titles of the top BLAST hits and their associated information will help you infer the species or at least narrow down the possibilities based on the sequence similarity to known sequences in the database.




In [22]:
from Bio.Blast import NCBIXML

blast_record = NCBIXML.read(result_handle)

# Print information about the top hits
for alignment in blast_record.alignments[:5]:  # Limit to top 5 hits for brevity
    for hsp in alignment.hsps:
        print("****Alignment****")
        print(f"sequence: {alignment.title}")
        print(f"length: {alignment.length}")
        print(f"e value: {hsp.expect}")
        print(f"identities: {hsp.identities}")


****Alignment****
sequence: gi|1783584753|gb|MN651324.1| Nicotiana tabacum strain zhongyan90 cytoplasmic male sterility(CMS) line cultivar MSzhongyan90 mitochondrion, complete genome
length: 530869
e value: 2.80689e-95
identities: 200
****Alignment****
sequence: gi|1783584659|gb|MN651323.1| Nicotiana tabacum strain zhongyan90 maintainer line cultivar zhongyan90 mitochondrion, complete genome
length: 472218
e value: 2.80689e-95
identities: 200
****Alignment****
sequence: gi|1783584536|gb|MN651322.1| Nicotiana tabacum strain yunyan85 cytoplasmic male sterility(CMS) line cultivar MSyunyan85 mitochondrion, complete genome
length: 468288
e value: 2.80689e-95
identities: 200
****Alignment****
sequence: gi|1783584454|gb|MN651321.1| Nicotiana tabacum cultivar Yunyan85 mitochondrion, complete genome
length: 430974
e value: 2.80689e-95
identities: 200
****Alignment****
sequence: gi|1741436215|gb|MH168703.1| Nicotiana tabacum/Hyoscyamus niger cybrid isolate rcMv-1-1g_c2 mitochondrion, complete se

# protein translation

To create a Biopython Seq object with the given DNA sequence and then find its **protein translation**, you need to follow these steps:

Create the Seq object: This involves initializing the Seq object with the provided DNA sequence.

Translate the sequence: Use the translate() method to translate the DNA sequence into its corresponding protein sequence. This method automatically handles the translation according to the standard genetic code, but you can specify alternative genetic codes if necessary.

Here is how you can do it:

In [24]:
from Bio.Seq import Seq

# Combine the given sequence into one continuous string, removing any newline characters or spaces
dna_sequence = """
TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTAC
AATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCAC
CTACGGTAGAG
""".replace('\n', '').replace(' ', '')

# Create the Seq object
my_seq = Seq(dna_sequence)

# Translate the DNA sequence to a protein sequence
protein_seq = my_seq.translate()

# Print the protein sequence
print("Protein Translation:", protein_seq)



Protein Translation: WASYLSYIPCSYGGAMFYVNPRSKDIIPKSYN*DLDMVLFCLSYRRLCSFLLFFGLLLILLWHLR*


/usr/local/lib/python3.10/dist-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
